# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [2]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/SequenceModelsNLP/FakeNewChallenge/RequiredData/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
import keras

In [0]:
train_bodies = pd.read_csv("/content/drive/My Drive/SequenceModelsNLP/FakeNewChallenge/RequiredData/train_bodies.csv")
train_stances = pd.read_csv("/content/drive/My Drive/SequenceModelsNLP/FakeNewChallenge/RequiredData/train_stances.csv")

In [33]:
train_bodies.shape

(1683, 2)

In [34]:
train_stances.shape

(49972, 3)

In [35]:
train_bodies.head(3)

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...


In [36]:
train_stances.head(3)

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated


In [0]:
dataset = pd.merge(train_bodies, train_stances, on="Body ID")


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [38]:
dataset.shape

(49972, 4)

In [39]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [41]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [0]:
from keras.preprocessing.text import Tokenizer

#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None, document_count=0)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
tokenizer.fit_on_texts(dataset["articleBody"])

In [0]:
tokenizer.fit_on_texts(dataset["Headline"])

In [46]:
len(tokenizer.word_counts)

27873

In [47]:
tokenizer.document_count

99944

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = list(dataset["articleBody"])


In [0]:
articles = []
for text in texts:
  articles.append(nltk.sent_tokenize(text))

## Check 2:

first element of texts and articles should be as given below. 

In [50]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [51]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

In [0]:
data = np.zeros((len(texts),MAX_SENTS, MAX_SENT_LENGTH))
for art_idx, article in enumerate(articles):
  for sent_idx, sent in enumerate(article):
    if sent_idx > MAX_SENTS-1:
        break
    cur_words = text_to_word_sequence(sent)
    for word_idx, word in enumerate(cur_words):
      if word_idx > MAX_SENT_LENGTH-1:
        break
      data[art_idx][sent_idx][word_idx] = tokenizer.word_index[word]

### Check 3:

Accessing first element in data should give something like given below.

In [54]:
data[0, :, :]

array([[3.0000e+00, 4.8100e+02, 4.2700e+02, 7.2110e+03, 8.1000e+01,
        3.0000e+00, 3.7330e+03, 3.3100e+02, 5.0000e+00, 3.8910e+03,
        3.5000e+02, 4.0000e+00, 1.4310e+03, 2.9580e+03, 1.0000e+00,
        8.9000e+01, 1.2000e+01, 4.6400e+02, 0.0000e+00, 0.0000e+00],
       [7.5800e+02, 9.5000e+01, 1.0470e+03, 3.0000e+00, 2.6790e+03,
        1.7520e+03, 7.0000e+00, 1.8900e+02, 3.0000e+00, 1.2170e+03,
        1.0750e+03, 2.0300e+03, 7.0000e+02, 1.5900e+02, 1.0000e+00,
        3.0320e+03, 4.4800e+02, 1.0000e+00, 5.5500e+02, 2.3500e+02],
       [8.9000e+01, 1.0670e+03, 4.1150e+03, 2.3490e+03, 1.2000e+01,
        3.0000e+00, 1.0920e+03, 3.3060e+03, 1.9000e+01, 1.0000e+00,
        8.9000e+01, 2.0000e+00, 1.7930e+03, 1.0000e+00, 5.2100e+02,
        2.0090e+03, 1.5000e+01, 9.0000e+00, 3.0000e+00, 3.1110e+03],
       [1.8100e+02, 3.6400e+03, 9.7200e+02, 2.0000e+02, 2.5560e+03,
        4.4000e+01, 6.7750e+03, 1.7220e+03, 1.2520e+03, 5.0000e+00,
        1.3317e+04, 1.7936e+04, 1.0000e+00, 7

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_heading = list(dataset["Headline"])
articles_heading = []
for text in texts:
  articles_heading.append(nltk.sent_tokenize(text))
  

data_heading = np.zeros((len(texts),MAX_SENTS_HEADING, MAX_SENT_LENGTH))
for art_idx, article in enumerate(articles_heading):
  for sent_idx, sent in enumerate(article):
    if sent_idx > MAX_SENTS_HEADING-1:
        break
    cur_words = text_to_word_sequence(sent)
    for word_idx, word in enumerate(cur_words):
      if word_idx > MAX_SENT_LENGTH-1:
        break
      data_heading[art_idx][sent_idx][word_idx] = tokenizer.word_index[word]


### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(dataset["Stance"])

In [0]:
labels = np.array(labels)

### Check 4:

The shape of data and labels shoould match the given below numbers.

In [61]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split

In [0]:
x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2)
x_heading_train, x_heading_val, y_train, y_val = train_test_split(data_heading, labels, test_size=0.2)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [66]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


In [76]:
X_train = np.reshape(x_train, (x_train.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_train.shape)

X_val = np.reshape(x_val, (x_val.shape[0], MAX_SENTS * MAX_SENT_LENGTH))
print(X_val.shape)

print(y_train.shape)
print(y_val.shape)

(39977, 400)
(9995, 400)
(39977, 4)
(9995, 4)


In [77]:
X_heading_train = np.reshape(x_heading_train, (x_heading_train.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_heading_train.shape)

X_heading_val = np.reshape(x_heading_val, (x_heading_val.shape[0], MAX_SENTS_HEADING * MAX_SENT_LENGTH))
print(X_heading_val.shape)

print(y_train.shape)
print(y_val.shape)

(39977, 20)
(9995, 20)
(39977, 4)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [78]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
vocab_size = len(tokenizer.word_counts)
embedding_matrix = np.zeros((vocab_size, 100))
t = tokenizer

for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
from keras.models import Model, Sequential
from keras.layers import Dense, BatchNormalization, Input, Concatenate, concatenate,Activation, Flatten, Dropout, Embedding, LSTM, Bidirectional, GlobalMaxPool1D, merge
from keras.optimizers import SGD

### Model

In [0]:
input1 = Input(shape=(MAX_SENTS * MAX_SENT_LENGTH, ))
input2 = Input(shape=(MAX_SENTS_HEADING * MAX_SENT_LENGTH, ))
embd1 = Embedding(len(tokenizer.word_counts), 100, weights=[embedding_matrix])(input1)
embd2 = Embedding(len(tokenizer.word_counts), 100, weights=[embedding_matrix])(input2)# embed size is 100 given that we've used glove.6B.100d.txt

biDirLSTM1 = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(embd1)
biDirLSTM2 = Bidirectional(LSTM(100, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(embd2)

maxPoolLayer1 = GlobalMaxPool1D()(biDirLSTM1)
maxPoolLayer2 = GlobalMaxPool1D()(biDirLSTM2)

dense1 = Dense(100, activation="relu")(maxPoolLayer1)
dp1 = Dropout(0.25)(dense1)

dense2 = Dense(100, activation="relu")(maxPoolLayer2)
dp2 = Dropout(0.25)(dense2)

x = concatenate([dp1, dp2])
x = Dense(4, activation="softmax")(x)
model = Model(inputs=[input1,input2], outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [82]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 400)          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 400, 100)     2787300     input_3[0][0]                    
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 20, 100)      2787300     input_4[0][0]                    
____________________________________________________________________________________________

### Compile and fit the model

In [0]:
from keras.utils import plot_model 

In [84]:
model.fit([X_train,X_heading_train], y_train, batch_size=64, epochs=5, validation_data=([X_val,X_heading_val], y_val))

Train on 39977 samples, validate on 9995 samples
Epoch 1/5
39977/39977 [==============================] - 1016s 25ms/step - loss: 0.3238 - acc: 0.8690 - val_loss: 0.2890 - val_acc: 0.8852
Epoch 2/5
39977/39977 [==============================] - 1025s 26ms/step - loss: 0.2725 - acc: 0.8914 - val_loss: 0.2561 - val_acc: 0.8997
Epoch 3/5
39977/39977 [==============================] - 1026s 26ms/step - loss: 0.2480 - acc: 0.9016 - val_loss: 0.2470 - val_acc: 0.9047
Epoch 4/5
39977/39977 [==============================] - 1025s 26ms/step - loss: 0.2349 - acc: 0.9057 - val_loss: 0.2408 - val_acc: 0.9048
Epoch 5/5
39977/39977 [==============================] - 998s 25ms/step - loss: 0.2290 - acc: 0.9076 - val_loss: 0.2389 - val_acc: 0.9068


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)